In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [3]:
spark = SparkSession.builder.appName("Dr Isaac Learning Class").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

#1 Leia o arquivo csv e transforme em dataframe

Dica: use o argumento `header=True`

In [5]:
amazon_users_csv = spark.read.csv('/content/amazon_prime_users.csv', header='True', inferSchema='True')

amazon_users_csv.printSchema()

root
 |-- User ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email Address: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Membership Start Date: date (nullable = true)
 |-- Membership End Date: date (nullable = true)
 |-- Subscription Plan: string (nullable = true)
 |-- Payment Information: string (nullable = true)
 |-- Renewal Status: string (nullable = true)
 |-- Usage Frequency: string (nullable = true)
 |-- Purchase History: string (nullable = true)
 |-- Favorite Genres: string (nullable = true)
 |-- Devices Used: string (nullable = true)
 |-- Engagement Metrics: string (nullable = true)
 |-- Feedback/Ratings: double (nullable = true)
 |-- Customer Support Interactions: integer (nullable = true)



In [19]:
amazon_users_csv.show()

+-------+-----------------+--------------------+------------------+-------------+------+-------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|User ID|             Name|       Email Address|          Username|Date of Birth|Gender|           Location|Membership Start Date|Membership End Date|Subscription Plan|Payment Information|Renewal Status|Usage Frequency|Purchase History|Favorite Genres|Devices Used|Engagement Metrics|Feedback/Ratings|Customer Support Interactions|
+-------+-----------------+--------------------+------------------+-------------+------+-------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|   

# 2 Mostre os 5 primeiros registros do dataframe

In [15]:
amazon_users_csv.head(5)

[Row(User ID=1, Name='Ronald Murphy', Email Address='williamholland@example.com', Username='williamholland', Date of Birth=datetime.date(1953, 6, 3), Gender='Male', Location='Rebeccachester', Membership Start Date=datetime.date(2024, 1, 15), Membership End Date=datetime.date(2025, 1, 14), Subscription Plan='Annual', Payment Information='Mastercard', Renewal Status='Manual', Usage Frequency='Regular', Purchase History='Electronics', Favorite Genres='Documentary', Devices Used='Smart TV', Engagement Metrics='Medium', Feedback/Ratings=3.6, Customer Support Interactions=3),
 Row(User ID=2, Name='Scott Allen', Email Address='scott22@example.org', Username='scott22', Date of Birth=datetime.date(1978, 7, 8), Gender='Male', Location='Mcphersonview', Membership Start Date=datetime.date(2024, 1, 7), Membership End Date=datetime.date(2025, 1, 6), Subscription Plan='Monthly', Payment Information='Visa', Renewal Status='Manual', Usage Frequency='Regular', Purchase History='Electronics', Favorite Ge

# 3 Conte a quantidade de registros (linhas) no dataframe

In [16]:
amazon_users_csv.count()

2500

# 4 Quantos usuários únicos estao presentes no dataframe?

In [18]:
amazon_users_csv.distinct().count()

2500

# 5 Qual o usuário mais velho presente no dataframe? (mais velho é diferente de mais antigo)

In [31]:
amazon_users_csv.createOrReplaceTempView("amazon_table")

In [33]:
spark.sql("SELECT * FROM amazon_table ORDER BY `Date of Birth` LIMIT 1;").show()

+-------+-------------+-------------------+--------+-------------+------+------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|User ID|         Name|      Email Address|Username|Date of Birth|Gender|          Location|Membership Start Date|Membership End Date|Subscription Plan|Payment Information|Renewal Status|Usage Frequency|Purchase History|Favorite Genres|Devices Used|Engagement Metrics|Feedback/Ratings|Customer Support Interactions|
+-------+-------------+-------------------+--------+-------------+------+------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|    800|Angela Barnes|laura45@example.net| laura45|

# 6 Qual a média do feedback agregada para cada genero no dataframe?

In [36]:
spark.sql("SELECT Gender, AVG(`Feedback/Ratings`) AS media_feedback FROM amazon_table GROUP BY Gender;").show()

+------+-----------------+
|Gender|   media_feedback|
+------+-----------------+
|Female|4.000967741935483|
|  Male|4.008492063492064|
+------+-----------------+



# 7 Defina um novo dataframe que possua apenas os registros possuindo mais do que 3 interações com o suporte de cliente

In [38]:
df_interacoes_suporte = spark.sql("SELECT * FROM amazon_table WHERE `Customer Support Interactions` > 3;")

In [39]:
df_interacoes_suporte.show()

+-------+-----------------+--------------------+-------------+-------------+------+-------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|User ID|             Name|       Email Address|     Username|Date of Birth|Gender|           Location|Membership Start Date|Membership End Date|Subscription Plan|Payment Information|Renewal Status|Usage Frequency|Purchase History|Favorite Genres|Devices Used|Engagement Metrics|Feedback/Ratings|Customer Support Interactions|
+-------+-----------------+--------------------+-------------+-------------+------+-------------------+---------------------+-------------------+-----------------+-------------------+--------------+---------------+----------------+---------------+------------+------------------+----------------+-----------------------------+
|      2|      Scot

# 8 A partir do dataframe definido anteriormente, defina outro que possua apenas as colunas 'Customer Support Interactions', 'Feedback/Ratings', 'User ID'

In [41]:
df_interacoes_feedbacks_id = spark.sql("SELECT `User ID`, `Customer Support Interactions`, `Feedback/Ratings` FROM amazon_table;")
df_interacoes_feedbacks_id.show()

+-------+-----------------------------+----------------+
|User ID|Customer Support Interactions|Feedback/Ratings|
+-------+-----------------------------+----------------+
|      1|                            3|             3.6|
|      2|                            7|             3.8|
|      3|                            8|             3.3|
|      4|                            7|             3.3|
|      5|                            1|             4.3|
|      6|                            2|             3.8|
|      7|                           10|             4.4|
|      8|                            6|             3.6|
|      9|                            8|             4.0|
|     10|                            6|             4.9|
|     11|                            1|             4.0|
|     12|                            6|             4.4|
|     13|                            0|             4.7|
|     14|                            0|             3.8|
|     15|                      

# 9 A partir do dataframe definido na questao anterior, converta as colunas 'Customer Support Interactions', 'Feedback/Ratings' de string para float  

In [51]:
df_interacoes_feedbacks_id = df_interacoes_feedbacks_id.withColumn("Customer Support Interactions", col("Customer Support Interactions").cast("double"))

In [53]:
df_interacoes_feedbacks_id = df_interacoes_feedbacks_id.withColumn("Feedback/Ratings", col("Feedback/Ratings").cast("double"))

In [54]:
df_interacoes_feedbacks_id.dtypes

[('User ID', 'int'),
 ('Customer Support Interactions', 'double'),
 ('Feedback/Ratings', 'double')]

# 10 A partir do dataframe definido na questao anterior, aplique padronizacao nas colunas 'Customer Support Interactions','Feedback/Ratings'
